# 6.1 重构“策略”模式

假如一个网店指定了下述折扣规则
* 有 1000 或以上积分的顾客，每个订单享 5% 的折扣
* 同一个订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
* 订单中的不同商品达到 10 个或以上，享 7% 折扣

简单起见，我们假定一个订单只能享用一次折扣
名词解释：
* 上下文    把一些计算委托给不同算法的可交互组件，它提供服务。在这个示例中，上下文是 Order，他会根据不同的算法计算折扣
* 策略      实现不同算法的组件的共同接口。在这个示例中，名为 Promotion 的抽象类扮演这个角色
* 具体策略  “策略” 的具体子类。 fidelityPromo、BulkPromo 和 LargeOrderPromo 是这里实现的三个具体策略

In [5]:
# 实现 Order 类
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price) -> None:
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order: # 上下文

    def __init__(self, customer, cart, promotion = None) -> None:
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        fmt = '<Order total: {:.2f} due : {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC):   # 策略:抽象基类

    @abstractmethod
    def discount(self, order):
        """返回折扣金额(正值)"""

class FidelityPromo(Promotion): # 第一个具体策略
    """为积分为 1000 及以上的用户提供 5% 折扣"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): # 第二个具体策略
    """单个商品 20 个及以上时提供 10% 折扣"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion):   # 第三个具体策略
    """订单中的不同商品达到 10 个或以上时提供 7% 折扣"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [6]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due : 42.00>

In [7]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due : 39.90>

In [8]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due : 28.50>

In [9]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due : 9.30>

In [12]:
# 使用函数实现”策略“模式
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price) -> None:
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:

    def __init__(self, customer, cart, promotion = None) -> None:
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # 此处直接调用函数
        return self.total() - discount
    
    def __repr__(self) -> str:
        fmt = '<Order total: {:.2f} due : {:.2f}>'
        return fmt.format(self.total(), self.due())

# 没有使用抽象基类

def fidelity_promo(order): # 直接定义函数而不是类
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

# 把函数当作参数传入而不是实例当参数
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due : 42.00>

In [21]:
# 选择最佳策略：简单的方式
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """选择最佳折扣"""
    return max(promo(order) for promo in promos)

In [17]:
# 选择 large_order_promo
Order (joe, long_order, best_promo)

<Order total: 30.00 due : 28.50>

In [ ]:
# 选择 bulk_item_promo
Order (joe, banana_cart, best_promo)

In [18]:
# 选择 fidelity_promo
Order (ann, cart, best_promo)

<Order total: 42.00 due : 39.90>

##### 找出模块中的全部策略
在上面的实现方式中，如果添加新策略那就需要改动 promos 列表，但我们也可以使用 globale() 来找出可用的 *_promo 函数  

__globals()__  
    返回一个字典，表示当前的全局符号表。这个符号表始终针对当前模块(对函数或方法来说，是指定义它们的模块，而不是调用它们的模块)

In [23]:
# 内省模块的命名空间构建 promos 列表
promos = [globals()[name] for name in globals()
            if name.endswith('_promo')
            and name != 'best_promo']

def best_promo(order):
    """选择最佳折扣"""
    return max(promo(order) for promo in promos)

Order (joe, long_order, best_promo)

<Order total: 10.00 due : 9.30>

In [32]:
print([globals()[name] for name in globals() if name.endswith('_promo')])

[<function fidelity_promo at 0x000001BD61645E50>, <function bulk_item_promo at 0x000001BD61649280>, <function large_order_promo at 0x000001BD61649310>, <function best_promo at 0x000001BD61649D30>]


In [ ]:
# 也可以使用 inspect 内省名为 promotions(promos都定义在这个模块)
import inspect
promos= [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

# inspect.getmembers 用于获取函数对象的属性，第二个参数是可选的判断条件(一个 bool 函数)，此处使用 inspect.isfunction，只获取其中的函数